In [ ]:
# import required libraries

import os
import sys
import configparser

from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision
from torchvision.transforms import RandomAffine
from torchvision.utils import make_grid

In [ ]:
# set the device

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device is:', device)

In [ ]:
# set the hyperparameters

config = configparser.ConfigParser()

config['CAE'] = {'rand_seed': 765, 'ks': 3, 'nf0': 15, 'nf1': 45, 'nf2': 128, 'nf3': 196, 'nf4': 128, 'nf5': 128, 'nf6': 128,
                 'nfc': 14}

config['IM'] = {'lambda_affine': 0.03, 'lambda_marginal_entropy': 0.1, 'lambda_conditional_entropy': 0.03,
                'learning_rate': 0.003}

config.write(sys.stdout)

In [ ]:
rand_seed = int(config['CAE']['rand_seed'])
ks        = int(config['CAE']['ks'])
nf0       = int(config['CAE']['nf0'])
nf1       = int(config['CAE']['nf1'])
nf2       = int(config['CAE']['nf2'])
nf3       = int(config['CAE']['nf3'])
nf4       = int(config['CAE']['nf4'])
nf5       = int(config['CAE']['nf5'])
nf6       = int(config['CAE']['nf6'])
nfc       = int(config['CAE']['nfc'])

In [ ]:
lambda_affine              = float(config['IM']['lambda_affine'])
lambda_marginal_entropy    = float(config['IM']['lambda_marginal_entropy'])
lambda_conditional_entropy = float(config['IM']['lambda_conditional_entropy'])
learning_rate              = float(config['IM']['learning_rate'])

In [ ]:
# set random seed

print('random seed:', rand_seed)

torch.manual_seed(rand_seed)
torch.cuda.manual_seed(rand_seed)
np.random.seed(rand_seed)

In [ ]:
# load the dataset

data_src = np.load('/project/dsc-is/mahfujul-r/M/slice64_Block2_20K.npy')

In [ ]:
# custom functions to extract batches of samples

def get_batch_index_tr(tr, batch_size=None, shuffle=True):
    if (shuffle):
        np.random.shuffle(tr)
    if (batch_size is not None):
        n_batch = len(tr) // batch_size
    batch_list = np.array_split(tr, n_batch)
    return batch_list

def get_batch_index_ae(ae, batch_size=None, shuffle=True):
    tr = np.arange(ae)
    batch_list = get_batch_index_tr(tr, batch_size=batch_size, shuffle=shuffle)
    return batch_list

In [ ]:
ix, iy = 64, 64 # 64x64 patches are not getting rescaled like 128x128 patches

#affine transformation
add_random_affine = RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05), fill=(161, 138, 172)) 

def generate_batch(iii, src, device, random_affine=False):
    if (random_affine):
        tmp = np.empty((len(iii), ix, iy, nf0))
        for aa, ii in enumerate(iii):
            img_tmp0 = Image.fromarray(src[ii, 0])
            img_tmp0 = add_random_affine(img_tmp0) # HE
            img_tmp4 = Image.fromarray(src[ii, 4])
            img_tmp4 = add_random_affine(img_tmp4) # CD31
            img_tmp5 = Image.fromarray(src[ii, 5])
            img_tmp5 = add_random_affine(img_tmp5) # CK19
            img_tmp6 = Image.fromarray(src[ii, 6])
            img_tmp6 = add_random_affine(img_tmp6) # Ki67
            img_tmp7 = Image.fromarray(src[ii, 7])
            img_tmp7 = add_random_affine(img_tmp7) # MT
            tmp[aa] = np.concatenate((img_tmp0, img_tmp4, img_tmp5, img_tmp6, img_tmp7), axis=2)
        xxx = torch.tensor(tmp/255.0, dtype=torch.float32).permute(0, 3, 2, 1)
    
    else:
        tmp = np.empty((len(iii), ix, iy, nf0))
        for aa, ii in enumerate(iii):
            img_tmp0 = Image.fromarray(src[ii, 0]) # HE
            img_tmp4 = Image.fromarray(src[ii, 4]) # CD31
            img_tmp5 = Image.fromarray(src[ii, 5]) # CK19
            img_tmp6 = Image.fromarray(src[ii, 6]) # Ki67
            img_tmp7 = Image.fromarray(src[ii, 7]) # MT
            tmp[aa] = np.concatenate((img_tmp0, img_tmp4, img_tmp5, img_tmp6, img_tmp7), axis=2)
        xxx = torch.tensor(tmp/255.0, dtype=torch.float32).permute(0, 3, 2, 1)
    
    return (xxx.to(device))

`CAE` (Convolutional AutoEncoder) architecture

- `Encoder` class

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential()
        self.encoder.add_module('conv1', nn.Conv2d(in_channels=nf0, out_channels=nf1, kernel_size=4, stride=2, padding=1))
        self.encoder.add_module('bnor1', nn.BatchNorm2d(num_features=nf1, affine=True, track_running_stats=True))
        self.encoder.add_module('lrel1', nn.LeakyReLU(0.1, inplace=True))
        self.encoder.add_module('conv2', nn.Conv2d(in_channels=nf1, out_channels=nf2, kernel_size=4, stride=2, padding=1))
        self.encoder.add_module('bnor2', nn.BatchNorm2d(num_features=nf2, affine=True, track_running_stats=True))
        self.encoder.add_module('lrel2', nn.LeakyReLU(0.1, inplace=True))
        self.encoder.add_module('conv3', nn.Conv2d(in_channels=nf2, out_channels=nf3, kernel_size=4, stride=2, padding=1))
        self.encoder.add_module('bnor3', nn.BatchNorm2d(num_features=nf3, affine=True, track_running_stats=True))
        self.encoder.add_module('lrel3', nn.LeakyReLU(0.1, inplace=True))
        self.encoder.add_module('conv4', nn.Conv2d(in_channels=nf3, out_channels=nf4, kernel_size=4, stride=2, padding=1))
        self.encoder.add_module('bnor4', nn.BatchNorm2d(num_features=nf4, affine=True, track_running_stats=True))
        self.encoder.add_module('lrel4', nn.LeakyReLU(0.1, inplace=True))
        
    def forward(self, xxx):
        hhh = self.encoder(xxx)        
        return hhh

- `Classifier` class

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.classifier = nn.Sequential()
        self.classifier.add_module('conv1', nn.Conv2d(in_channels=nf4, out_channels=nf5, kernel_size=4, stride=1, padding=0))
        self.classifier.add_module('bnor1', nn.BatchNorm2d(num_features=nf5, affine=True, track_running_stats=True))
        self.classifier.add_module('lrel1', nn.LeakyReLU(0.1, inplace=True))
        self.classifier.add_module('conv2', nn.Conv2d(in_channels=nf5, out_channels=nf6, kernel_size=1, stride=1, padding=0))
        self.classifier.add_module('lrel2', nn.LeakyReLU(0.1, inplace=True))
        self.classifier.add_module('conv3', nn.Conv2d(in_channels=nf6, out_channels=nfc, kernel_size=1, stride=1, padding=0))
        self.classifier.add_module('lrel3', nn.LeakyReLU(0.1, inplace=True))
        
    def forward(self, hhh):
        vvv = self.classifier(hhh)
        return vvv

- `Decoder` class

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        self.decoder = nn.Sequential()
        self.decoder.add_module('upsm4', nn.UpsamplingBilinear2d(scale_factor=2))
        self.decoder.add_module('dcov4', nn.Conv2d(in_channels=nf4 + nfc, out_channels=nf3, kernel_size=3, stride=1, padding=1))
        self.decoder.add_module('norm4', nn.BatchNorm2d(num_features=nf3, affine=True, track_running_stats=True))
        self.decoder.add_module('lrel4', nn.LeakyReLU(0.1, inplace=True))
        self.decoder.add_module('upsm3', nn.UpsamplingBilinear2d(scale_factor=2))
        self.decoder.add_module('dcov3', nn.Conv2d(in_channels=nf3, out_channels=nf2, kernel_size=3, stride=1, padding=1))
        self.decoder.add_module('norm3', nn.BatchNorm2d(num_features=nf2, affine=True, track_running_stats=True))
        self.decoder.add_module('lrel3', nn.LeakyReLU(0.1, inplace=True))
        self.decoder.add_module('upsm2', nn.UpsamplingBilinear2d(scale_factor=2))
        self.decoder.add_module('dcov2', nn.Conv2d(in_channels=nf2, out_channels=nf1, kernel_size=3, stride=1, padding=1))
        self.decoder.add_module('norm2', nn.BatchNorm2d(num_features=nf1, affine=True, track_running_stats=True))
        self.decoder.add_module('lrel2', nn.LeakyReLU(0.1, inplace=True))
        self.decoder.add_module('upsm1', nn.UpsamplingBilinear2d(scale_factor=2))
        self.decoder.add_module('dcov1', nn.Conv2d(in_channels=nf1, out_channels=nf0, kernel_size=3, stride=1, padding=1))
        self.decoder.add_module('norm1', nn.BatchNorm2d(num_features=nf0, affine=True, track_running_stats=True))
        self.decoder.add_module('sgmd1', nn.Sigmoid())
        
    def forward(self, hhh, vvv):
        ccc = vvv.repeat((1, 1, ix//16, iy//16))
        hhh = torch.cat((hhh, ccc), dim=1)
        yyy = self.decoder(hhh)
        return yyy

In [ ]:
model_en = Encoder().to(device)
model_cl = Classifier().to(device)
model_de = Decoder().to(device)

# load models
model_en.load_state_dict(torch.load('models14/model_encoder_3000'))
model_cl.load_state_dict(torch.load('models14/model_classifier_3000'))
model_de.load_state_dict(torch.load('models14/model_decoder_3000'))

In [ ]:
iii_batch = get_batch_index_ae(15000, batch_size=100, shuffle=False) # first 15000 samples

hhh_list = []

with torch.inference_mode():
    for iii in iii_batch:
        xxx_tmp = generate_batch(iii, data_src, device)
        
        model_en.eval()
        
        hhh_tmp = model_en(xxx_tmp)
        
        hhh_list.append(torch.softmax(hhh_tmp, dim=1).detach().cpu().numpy())
        
    hhh = np.concatenate(hhh_list, axis=0)
    print('Shape of upper latent space:', hhh.shape)

In [ ]:
a = np.mean(hhh, axis=3)
a = np.mean(a, axis=2)
a.shape # embedded features in upper latent space

In [ ]:
iii_batch = get_batch_index_ae(15000, batch_size=100, shuffle=False) # first 15000 samples

vvv_list = []

with torch.inference_mode():
    for iii in iii_batch:
        xxx_tmp = generate_batch(iii, data_src, device)
        
        model_en.eval()
        model_cl.eval()
        
        hhh_tmp = model_en(xxx_tmp)
        vvv_tmp = model_cl(hhh_tmp)
        
        vvv_list.append(torch.softmax(vvv_tmp.reshape((-1, nfc)), dim=1).detach().cpu().numpy())
        
    vvv_15k = np.concatenate(vvv_list, axis=0)
    print('Embedded features in lower latent space:', vvv_15k.shape)

In [ ]:
C = []
D = []

for j in range(len(vvv_15k)):
    hemp1 = vvv_15k[j, :]
    hemp2 = np.argmax(hemp1)
    hemp3 = hemp1[hemp2]
    C.append(hemp2) # collect cluster IDs with highest probability for each patch
    D.append(hemp3) # collect highest probability of each patch

In [ ]:
DF = pd.DataFrame(a)
DF['128'] = C
DF['129'] = D
DF # this dataframe contains embedded features from upper latent space, cluster IDs and highest probability (of each patch) 

In [ ]:
# 'HHH14 & C14 & D14.csv' file will be saved in the present working directory

DF.to_csv('HHH14 & C14 & D14.csv', index=None, header=None) # save DF as a '.csv' file

In [ ]:
# clustering

fig, plts = plt.subplots(nrows=14, ncols=1)

for cc in range(nfc):
    iii = np.argsort(-vvv_15k[:, cc])
    
    tmp_p1 = []
    tmp_p2 = []
    tmp_p3 = []
    tmp_p4 = []
    tmp_p5 = []
    
    for aa in range(8):
        tmp_p1.append(torch.from_numpy(data_src[iii[aa], 0]).permute(2, 0, 1))
        tmp_p2.append(torch.from_numpy(data_src[iii[aa], 7]).permute(2, 0, 1))
        tmp_p3.append(torch.from_numpy(data_src[iii[aa], 4]).permute(2, 0, 1))
        tmp_p4.append(torch.from_numpy(data_src[iii[aa], 5]).permute(2, 0, 1))
        tmp_p5.append(torch.from_numpy(data_src[iii[aa], 6]).permute(2, 0, 1))
    
    grid = make_grid([tmp_p1[0], tmp_p1[1], tmp_p1[2], tmp_p1[3], tmp_p1[4], tmp_p1[5], tmp_p1[6], tmp_p1[7],
                      tmp_p2[0], tmp_p2[1], tmp_p2[2], tmp_p2[3], tmp_p2[4], tmp_p2[5], tmp_p2[6], tmp_p2[7],
                      tmp_p3[0], tmp_p3[1], tmp_p3[2], tmp_p3[3], tmp_p3[4], tmp_p3[5], tmp_p3[6], tmp_p3[7],
                      tmp_p4[0], tmp_p4[1], tmp_p4[2], tmp_p4[3], tmp_p4[4], tmp_p4[5], tmp_p4[6], tmp_p4[7],
                      tmp_p5[0], tmp_p5[1], tmp_p5[2], tmp_p5[3], tmp_p5[4], tmp_p5[5], tmp_p5[6], tmp_p5[7]],
                      nrow=8, padding=3)
    
    img = torchvision.transforms.ToPILImage()(grid)
    plts[cc].imshow(img)
    plts[cc].set_ylabel('Cluster {}'.format(cc + 1), fontsize=12)
    plts[cc].set_xticks([])
    plts[cc].set_yticks([])

fig.set_size_inches((10, 30))
fig.set_dpi(300)
fig.show()